In [27]:
include("../Algebra/Gradients.jl")
include("../Algebra/Hamiltonians.jl")
include("../Algebra/Matrices.jl")
include("../Algebra/Propagators.jl")
include("../Amplitudes/Chebyshev.jl")
include("../Costs/Costs.jl")
include("../Gates/Xgate.jl")
using LinearAlgebra, Optim, Plots
using BenchmarkTools, Distributions, DataFrames, CSV

In [34]:
ω_c = 5
ω_q = 2
χ = 3e-3
N = 8
amp_c = 1
amp_q = 1

1

In [35]:
I_qubit = Matrix(I,2,2)
I_cavity = Matrix(I,N,N)
#X_gate = kron(x_gate(N, Array[[3,4], [5,6], [7,8]]), I_qubit)
X_gate = kron(x_gate(N, Array[[3,4]]), I_qubit)
#X_gate = kron(x_gate(N, Array[[2,3],[4,5]]), I_qubit)

θ = π/2.5
RZ = kron(Rz(N, Array[[3,4]], θ/2), I_qubit)
RX = kron(Rx(N, Array[[3,4]], θ/4), I_qubit)
Udue = kron(U2(N, Array[[3,4]], [θ/2 θ/3 θ/4] ), I_qubit)
sx = kron(I_cavity, [0 1; 1 0])
sy = kron(I_cavity, [0 -1im; 1im 0])
# generating matrices
a,adag,sp,sm,sz = generate_matrices(N)

ψ_initial = zeros(N)
ψ_initial[1] = 1
ψ_initial = kron(ψ_initial/norm(ψ_initial), [0,1])

ψ_initial_rand = rand(N)
ψ_initial_rand = kron(ψ_initial_rand/norm(ψ_initial_rand), [0,1])
ψ_final_u = Udue * ψ_initial_rand
ψ_final_x = RX * ψ_initial_rand
ψ_final_x_gate = X_gate * ψ_initial_rand
ψ_final_superposition = kron(ones(N)/norm(ones(N)), [0,1])

ψ_final_k = zeros(N)
ψ_final_k[4] = 1
ψ_final_k = kron(ψ_final_k/norm(ψ_final_k), [0,1])

16-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [36]:
disp(b) = exp(b * adag - b * a)
snap(x) = kron(Diagonal(exp.(-1im.*x)),I_qubit)

ECD(β) = exp(β/2 * adag - β'/2 * a) * sp + exp(-β/2 * adag + β'/2 * a) * sm
ROT(ϕ, θ) = exp(-1im * θ / 2 * ( sx * cos(ϕ) + sy * sin(ϕ) )) 

ROT (generic function with 1 method)

In [37]:
function snap_disp_classic(coefficients, unitary, N, blocks,disp,snap)

    αsi = coefficients[1: blocks]
    θs = coefficients[blocks + 1 : end]
    propagator = Matrix{ComplexF64}(I,N,N)
    q_d = Int(N/2)

    for i in 1:blocks
        propagator = propagator * disp(αsi[i]) * snap(θs[(i - 1) * q_d + 1 : i * q_d])
    end

    return 1 - norm(tr(propagator*unitary') / N)^2
    
end

function snap_disp_classic_gate(coefficients, unitary, N, blocks,disp,snap)

    αsi = coefficients[1: blocks]
    θs = coefficients[blocks + 1 : end]
    propagator = Matrix{ComplexF64}(I,N,N)
    q_d = Int(N/2)

    for i in 1:blocks
        propagator = propagator * disp(αsi[i]) * snap(θs[(i - 1) * q_d + 1 : i * q_d])
    end

    return propagator
    
end

function snap_disp_cost_gate(coefficients, unitary, N, blocks,disp,snap)

    L = length(coefficients)
    αs = coefficients[1: blocks]
    q_d = Int(N/2)

    θs = coefficients[blocks + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:blocks
        propagator = propagator * disp(αs[i]) * snap(θs[(i - 1) * q_d + 1 : i * q_d]) * disp(αs[i])'
    end

    return 1 - norm(tr(propagator*unitary) / N)^2
    
end

function snap_disp_cost_gate_norm(coefficients, unitary, N, blocks,disp,snap)

    L = length(coefficients)
    αs = coefficients[1: blocks]
    q_d = Int(N/2)

    θs = coefficients[blocks + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:blocks
        propagator = propagator * disp(αs[i]) * snap(θs[(i - 1) * q_d + 1 : i * q_d]) * disp(αs[i])'
    end

    return norm(propagator - unitary)
    
end

function snap_disp_cost_gate_opnorm(coefficients, unitary, N, blocks,disp,snap)

    L = length(coefficients)
    αs = coefficients[1: blocks]
    q_d = Int(N/2)

    θs = coefficients[blocks + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:blocks
        propagator = propagator * disp(αs[i]) * snap(θs[(i - 1) * q_d + 1 : i * q_d]) * disp(αs[i])'
    end

    return opnorm(propagator - unitary)
    
end

function snap_disp_cost_state_prep(coefficients, N, ψ_n, ψ_m, blocks)


    αs = coefficients[1: blocks]

    θs = coefficients[blocks + 1 : end]

    q_d = Int(N/2)

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:blocks
        propagator = propagator * disp(αs[i]) * snap(θs[(i - 1) * q_d + 1 : i * q_d]) * disp(αs[i])'
    end

    return 1 - norm(ψ_m' * propagator * ψ_n)^2
    
end


snap_disp_cost_state_prep (generic function with 1 method)

## STATE PREP

In [38]:
it_costs_x = []
coefficients_x = []
it_costs_rx = []
coefficients_rx = []
it_costs_u = []
coefficients_u = []
it_costs_superposition = []
coefficients_superposition = []
it_costs_k = []
coefficients_k = []
blocks = []
for depth in 1:1:5
    x(x) = snap_disp_cost_state_prep(x, N * 2, ψ_initial_rand, ψ_final_u,depth)
    rx(x) = snap_disp_cost_state_prep(x, N * 2, ψ_initial_rand, ψ_final_x,depth)
    u(x) = snap_disp_cost_state_prep(x, N * 2, ψ_initial_rand, ψ_final_x_gate, depth)
    k(x) = snap_disp_cost_state_prep(x, N * 2, ψ_initial, ψ_final_k,depth)
    superposition(x) = snap_disp_cost_state_prep(x, N * 2, ψ_initial, ψ_final_superposition,depth)
    for iteration in 1:5
        println("blocks: ", depth," iterations :", iteration)
        coefficients = zeros((N + 1 ) * depth)
        coefficients[1: depth] = rand(depth)*2 .-1
        coefficients[depth + 1:end] = rand(N*depth) * 2 * pi .- pi
        res_x = Optim.optimize(x, coefficients , LBFGS(), Optim.Options())
        res_rx = Optim.optimize(rx, coefficients , LBFGS(), Optim.Options())
        res_u = Optim.optimize(u, coefficients , LBFGS(), Optim.Options())
        res_superposition = Optim.optimize(superposition, coefficients , LBFGS(), Optim.Options())
        res_k = Optim.optimize(k, coefficients , LBFGS(), Optim.Options())
        push!(blocks, depth)
        push!(it_costs_x,Optim.minimum(res_x))
        push!(it_costs_rx,Optim.minimum(res_rx))
        push!(it_costs_u,Optim.minimum(res_u))
        push!(it_costs_k,Optim.minimum(res_k))
        push!(it_costs_superposition,Optim.minimum(res_superposition))
        push!(coefficients_x,Optim.minimizer(res_x))
        push!(coefficients_rx,Optim.minimizer(res_rx))
        push!(coefficients_u,Optim.minimizer(res_u))
        push!(coefficients_k,Optim.minimizer(res_k))
        push!(coefficients_superposition,Optim.minimizer(res_superposition))
    end
end

blocks: 1 iterations :1
blocks: 

1 iterations :2


blocks: 1 iterations :3
blocks: 

1 iterations :4


blocks: 1 iterations :5
blocks: 

...

blocks: 5 iterations :1
blocks: 

5 iterations :2


blocks: 5 iterations :3
blocks: 

5 iterations :4


blocks: 5 iterations :5


In [39]:
df_x = DataFrame(fidelity=it_costs_x, blocks=blocks, coefficients=coefficients_x)
df_rx = DataFrame(fidelity=it_costs_rx, blocks=blocks, coefficients=coefficients_rx)
df_u = DataFrame(fidelity=it_costs_u, blocks=blocks, coefficients=coefficients_u)
df_k = DataFrame(fidelity=it_costs_k, blocks=blocks, coefficients=coefficients_k)
df_superposition = DataFrame(fidelity=it_costs_superposition, blocks=blocks, coefficients=coefficients_superposition)
CSV.write("../../data/state_prep/7_photons/snap_disp/x_gate.txt", df_x)
CSV.write("../../data/state_prep/7_photons/snap_disp/rx_gate.txt", df_rx)
CSV.write("../../data/state_prep/7_photons/snap_disp/u_gate.txt", df_u)
CSV.write("../../data/state_prep/7_photons/snap_disp/k_gate.txt", df_k)
CSV.write("../../data/state_prep/7_photons/snap_disp/superposition_gate.txt", df_superposition)

"../../data/state_prep/7_photons/snap_disp/superposition_gate.txt"

## trace

In [46]:
it_costs_x = []
coefficients_x = []
blocks_array = []
it_costs_rx = []
coefficients_rx = []
for blocks in 1:1:5
    for iteration in 1:10
        println("blocks: ", blocks," iterations :", iteration)
        coefficients_initial = zeros((N + 1 ) * blocks)
        coefficients_initial[1: blocks] = rand(blocks)*2 .-1
        coefficients_initial[blocks + 1:end] = rand(N*blocks) * 2 * pi .- pi
        f(x) = snap_disp_cost_gate(x, RX, N * 2, blocks, disp,snap)
        g(x) = snap_disp_cost_gate(x, X_gate, N * 2, blocks, disp,snap)
        res_x = Optim.optimize(g, coefficients_initial , LBFGS(), Optim.Options())
        res_rx = Optim.optimize(f, coefficients_initial , LBFGS(), Optim.Options())
        push!(it_costs_x,Optim.minimum(res_x))
        push!(it_costs_rx,Optim.minimum(res_rx))
        push!(coefficients_x,Optim.minimizer(res_x))
        push!(blocks_array, blocks)
        push!(coefficients_rx,Optim.minimizer(res_rx))
    end
end

blocks: 1 iterations :1
blocks: 

1 iterations :2
blocks: 1 iterations :3
blocks: 1 iterations :4


blocks: 1 iterations :5
blocks: 1 iterations :6
blocks: 1 iterations :7
blocks: 

1 iterations :8
blocks: 1 iterations :9
blocks: 1 iterations :10


blocks: 2 iterations :1
blocks: 

...

5 iterations :6


blocks: 5 iterations :7
blocks: 

5 iterations :8


blocks: 5 iterations :9
blocks: 

5 iterations :10


In [47]:
save = true
if save
    df_x = DataFrame(fidelity=it_costs_x, blocks=blocks_array, coefficients=coefficients_x)
    df_rx = DataFrame(fidelity=it_costs_rx, blocks=blocks_array, coefficients=coefficients_rx)
    CSV.write("../../data/(3,4).txt", df_x)
    CSV.write("../../data/pi_over_10.txt", df_rx)
end

"../../data/pi_over_10.txt"